In [4]:
from torch.utils.tensorboard import SummaryWriter
import torch                     
import torch.nn as nn           
import torch.nn.functional as F 
import torchvision
from torch import optim
from torch.utils.data import ConcatDataset 
import torchvision.transforms as transforms


In [34]:
transform = transforms.Compose([transforms.ToTensor()]) 
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [35]:
trainset[0][0].shape
x = torch.stack([sample[0] for sample in trainset])
x.shape

torch.Size([50000, 3, 32, 32])

In [36]:
mean = torch.mean(x,(0, 2, 3))
mean
std = torch.std(x,(0, 2, 3))
std

tensor([0.2470, 0.2435, 0.2616])

In [15]:
mean = torch.tensor([0.4914, 0.4822, 0.4465])
std = torch.tensor([0.2470, 0.2435, 0.2616])

In [16]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean, std)])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

100%|██████████| 170M/170M [00:26<00:00, 6.52MB/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data


In [7]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [19]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [27]:
lr = 0.001
device = 'cuda'
net = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr)
epochs = 2
writer = SummaryWriter(f'runs/{optimizer.__class__.__name__}_{lr}_{epochs}')

In [28]:
for epoch in range(epochs):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            writer.add_scalar('training loss',
                            running_loss / 2000,
                            epoch * len(trainloader) + i)
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.845
[1,  4000] loss: 1.623
[1,  6000] loss: 1.523
[1,  8000] loss: 1.502
[1, 10000] loss: 1.468
[1, 12000] loss: 1.430
[2,  2000] loss: 1.349
[2,  4000] loss: 1.353
[2,  6000] loss: 1.366
[2,  8000] loss: 1.312
[2, 10000] loss: 1.294
[2, 12000] loss: 1.282
Finished Training


In [44]:
images, labels = next(iter(trainloader))
writer.add_graph(net, images.to(device))
writer.close()

In [29]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = inputs.to(device), labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 10 %
